In [32]:
include("util.jl");

## Day 1

In [35]:
depths = getinput(1) |> split .|> int

function count_ascending(arr)
    (d′ > d for (d, d′) in zip(arr, arr[2:end])) |> sum
end;

In [37]:
@time count_ascending(depths)

  0.000008 seconds (2 allocations: 15.766 KiB)


1532

In [38]:
sums3 = (depths[i:i+2] for i ∈ 1:(length(depths)-2)) .|> sum
@time count_ascending(sums3)

  0.000011 seconds (2 allocations: 15.766 KiB)


1571

## Day 2

In [44]:
dirs = Dict(
    "forward" => [1, 0],
    "down" => [0, 1],
    "up" => [0, -1]
)

function parse_row(row)
    dir, mag = split(row)
    return dirs[dir] * int(mag)
end

commands = split(getinput(2), "\n") .|> parse_row

*(sum(commands)...)

2147104

In [49]:
pos = [0, 0]
aim = 0

for row in split(getinput(2), "\n")
    command, mag = split(row)
    if command == "forward"
       pos += [1, aim] * int(mag)
    else
        aim += (command == "down" ? 1 : -1) * int(mag)
    end    
end

pos, *(pos...)

([1832, 1116059], 2044620088)